<a href="https://colab.research.google.com/github/Praz98/Praz98-Task-5-which-factors-predict-future-user-adoption/blob/main/predction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import python_script as script
import pandas as pd
from pandas import DataFrame, Series
# Read in the data
activity = pd.read_csv('/content/takehome_user_engagement (1).csv')
users = pd.read_csv('/content/takehome_users.csv', encoding='latin-1')
# Convert timestamp
activity['time_stamp2'] = pd.to_datetime(activity['time_stamp'])
# Only keep users who have logged at least 3 days
repeated_users_df = script.keep_repeat_users(activity)
# Split data by user_id
grouped_users_df = repeated_users_df.groupby('user_id')
# Apply function to see if logged 3 separate days in 7 day period
unique_active_peeps_df = script.keep_active_users(grouped_users_df)
# Create an indicator variable if they are an adopted user or not
unique_active_peeps_df['adopted_user'] = 1
# Merge adopted user dataframe with original dataframe
adopted_user_info = pd.merge(unique_active_peeps_df, users, how='outer',
                  left_on='user_id', right_on='object_id')
# Fill non-adopted users in with 0
adopted_user_info['adopted_user'] = adopted_user_info['adopted_user'].fillna(0)
len(adopted_user_info)
12000
### Want to create a few additional variables regarding timing of account usage
import numpy as np
import time
temp = adopted_user_info
temp['creation_time_utc'] = pd.to_datetime(temp['creation_time'], utc=1)
temp['creation_time_unix'] = temp['creation_time_utc'].astype(np.int64) // 10 ** 9
# Time from account creation to last login
temp['creation_delta'] = temp['last_session_creation_time'] - temp['creation_time_unix']
# Time from account creation to today
temp['lifespan_delta'] = int(time.time()) - temp['creation_time_unix']
# Time from last login to today
temp['last_login_delta'] = int(time.time()) - temp['last_session_creation_time']
temp.to_csv('/content/adopted_users_extended.csv')